In [5]:
from ogb.nodeproppred import PygNodePropPredDataset
from torch_geometric.data import DataLoader
from torch_geometric.nn import MetaPath2Vec
import torch

In [25]:
dataset = PygNodePropPredDataset(name = "ogbn-mag")

# split_idx = dataset.get_idx_split()
# train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
# train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True)
# valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False)
# test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False)

graph = dataset[0]
graph

/zhome/a6/2/155003/deep_learning/final_project/project_venv/lib/python3.11/site-packages/ogb/nodeproppred/dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  se

Data(
  num_nodes_dict={
    author=1134649,
    field_of_study=59965,
    institution=8740,
    paper=736389,
  },
  edge_index_dict={
    (author, affiliated_with, institution)=[2, 1043998],
    (author, writes, paper)=[2, 7145660],
    (paper, cites, paper)=[2, 5416271],
    (paper, has_topic, field_of_study)=[2, 7505078],
  },
  x_dict={ paper=[736389, 128] },
  node_year={ paper=[736389, 1] },
  edge_reltype={
    (author, affiliated_with, institution)=[1043998, 1],
    (author, writes, paper)=[7145660, 1],
    (paper, cites, paper)=[5416271, 1],
    (paper, has_topic, field_of_study)=[7505078, 1],
  },
  y_dict={ paper=[736389, 1] }
)

In [26]:
del graph.edge_index_dict[('author', 'affiliated_with', 'institution')]
graph.edge_index_dict

{('author',
  'writes',
  'paper'): tensor([[      0,       0,       0,  ..., 1134647, 1134648, 1134648],
         [  19703,  289285,  311768,  ...,  657395,  671118,  719594]]),
 ('paper',
  'cites',
  'paper'): tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
         [    88,  27449, 121051,  ..., 421711, 427339, 439864]]),
 ('paper',
  'has_topic',
  'field_of_study'): tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
         [   145,   2215,   3205,  ...,  21458,  22283,  31934]])}

In [ ]:
metapath = [
    ('author', 'writes', 'paper'),
    ('paper', 'cites', 'paper'),
    ('paper', 'has_topic', 'field_of_study'),
    # ('author', 'affiliated_with', 'institution'),
]

if torch.cuda.is_available():
    device = torch.device('cuda')

model = MetaPath2Vec(graph.edge_index_dict, embedding_dim=128,
                     metapath=metapath, walk_length=3, context_size=4,
                     walks_per_node=5, num_negative_samples=5,
                     sparse=True).to(device)


loader = model.loader(batch_size=128, shuffle=True, num_workers=6)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


def train(epoch, log_steps=100, eval_steps=2000):
    model.train()

    total_loss = 0
    for i, (pos_rw, neg_rw) in enumerate(loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if (i + 1) % log_steps == 0:
            print(f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                  f'Loss: {total_loss / log_steps:.4f}')
            total_loss = 0

        if (i + 1) % eval_steps == 0:
            acc = test()
            print(f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                  f'Acc: {acc:.4f}')


@torch.no_grad()
def test(train_ratio=0.1):
    model.eval()

    z = model('paper', batch=graph.x_dict['paper'].to(device))
    y = graph.y_dict['paper']

    perm = torch.randperm(z.size(0))
    train_perm = perm[:int(z.size(0) * train_ratio)]
    test_perm = perm[int(z.size(0) * train_ratio):]

    return model.test(z[train_perm], y[train_perm], z[test_perm], y[test_perm],
                      max_iter=150)


for epoch in range(1, 6):
    train(epoch)
    acc = test()
    print(f'Epoch: {epoch}, Accuracy: {acc:.4f}')

Epoch: 1, Step: 00100/8865, Loss: 9.9194
Epoch: 1, Step: 00200/8865, Loss: 9.7962
Epoch: 1, Step: 00300/8865, Loss: 9.5744
Epoch: 1, Step: 00400/8865, Loss: 9.3742
Epoch: 1, Step: 00500/8865, Loss: 9.2839
Epoch: 1, Step: 00600/8865, Loss: 9.1937
Epoch: 1, Step: 00700/8865, Loss: 9.1473
Epoch: 1, Step: 00800/8865, Loss: 9.1229
Epoch: 1, Step: 00900/8865, Loss: 9.1325
Epoch: 1, Step: 01000/8865, Loss: 9.0791
Epoch: 1, Step: 01100/8865, Loss: 9.0311
Epoch: 1, Step: 01200/8865, Loss: 9.0386
Epoch: 1, Step: 01300/8865, Loss: 8.9932
Epoch: 1, Step: 01400/8865, Loss: 8.9473
Epoch: 1, Step: 01500/8865, Loss: 8.9716
Epoch: 1, Step: 01600/8865, Loss: 8.8883
Epoch: 1, Step: 01700/8865, Loss: 8.9272
Epoch: 1, Step: 01800/8865, Loss: 8.8569
Epoch: 1, Step: 01900/8865, Loss: 8.8382
Epoch: 1, Step: 02000/8865, Loss: 8.8433


AttributeError: 'dict' object has no attribute 'to'

In [39]:
graph.x_dict['paper']

tensor([[-0.0954,  0.0408, -0.2109,  ...,  0.0616, -0.0277, -0.1338],
        [-0.1510, -0.1073, -0.2220,  ...,  0.3458, -0.0277, -0.2185],
        [-0.1148, -0.1760, -0.2606,  ...,  0.1731, -0.1564, -0.2780],
        ...,
        [ 0.0228, -0.0865,  0.0981,  ..., -0.0547, -0.2077, -0.2305],
        [-0.2891, -0.2029, -0.1525,  ...,  0.1042,  0.2041, -0.3528],
        [-0.0890, -0.0348, -0.2642,  ...,  0.2601, -0.0875, -0.5171]])

In [10]:
metapath[:-1], metapath[1:]

([('author', 'affiliated_with', 'institution'),
  ('author', 'writes', 'paper'),
  ('paper', 'cites', 'paper')],
 [('author', 'writes', 'paper'),
  ('paper', 'cites', 'paper'),
  ('paper', 'has_topic', 'field_of_study')])

In [11]:
for edge_type1, edge_type2 in zip(metapath[:-1], metapath[1:]):
    # if edge_type1[-1] != edge_type2[0]:
    print(edge_type1[-1], edge_type2[0])

institution author
paper paper
paper paper
